In [1]:
# This repository is designed for use in Kaggle. If you would like to use it in Google
# Colab, please change requirements_kaggle.txt to requirements.txt

# Note: If the clone failed, please check at the right side bar if Session options > 
# Internet is on. You would need to verify your phone number if you are turning it 
# on for the first time.

# Remember to turn on Session options > Persistence > Files only, 
# else your files might be lost.

# Clone the git repository
!git clone https://github.com/IsaacSzeccl/APAI3010_Realfill.git

fatal: destination path 'APAI3010_Realfill' already exists and is not an empty directory.


In [1]:
%cd APAI3010_Realfill/realfill
%ls

/kaggle/working/APAI3010_Realfill/realfill
APAI3010_Realfill/  infer.py   README.md                requirements.txt
data/               LICENSE    realfill.zip             train_realfill.ipynb
flowerwoman-model/  model.zip  requirements_kaggle.txt  train_realfill.py


In [2]:
!pip install -r requirements_kaggle.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 46.2 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Turn on accelerator

from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [4]:
import os
# Building environment variables
# For training
os.environ["TRAIN_DIR"] = "data/flowerwoman"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["OUTPUT_DIR"] = "flowerwoman-model"
os.environ['PRECISION_ARG'] = '--mixed_precision=fp16'

# For inference
os.environ["VAL_IMG"] = "data/flowerwoman/target/target.png"
os.environ["VAL_MASK"] = "data/flowerwoman/target/mask.png"
os.environ["OUTPUT_IMG_DIR"] = "flowerwoman-results"

In [5]:
# For checking if the path is valid
!ls $TRAIN_DIR
!ls $OUTPUT_DIR
!ls $VAL_IMG
!ls $VAL_MASK
!mkdir $OUTPUT_IMG_DIR
!ls $OUTPUT_IMG_DIR


ref  target
checkpoint-100	 checkpoint-1600  checkpoint-400     logs
checkpoint-1000  checkpoint-1700  checkpoint-500     model_index.json
checkpoint-1100  checkpoint-1800  checkpoint-600     scheduler
checkpoint-1200  checkpoint-1900  checkpoint-700     text_encoder
checkpoint-1300  checkpoint-200   checkpoint-800     tokenizer
checkpoint-1400  checkpoint-2000  checkpoint-900     unet
checkpoint-1500  checkpoint-300   feature_extractor  vae
data/flowerwoman/target/target.png
data/flowerwoman/target/mask.png


In [8]:
import torch
torch.cuda.empty_cache()

In [16]:
# For training

!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  $PRECISION_ARG \
  --use_8bit_adam \
  --set_grads_to_none \
  --enable_xformers_memory_efficient_attention

^C
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 5, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/usr/local/lib/python3.11/dist-packages/accelerate/__init__.py", line 16, in <module>
    from .accelerator import Accelerator
  File "/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py", line 36, in <module>
    from accelerate.utils.imports import is_torchao_available
  File "/usr/local/lib/python3.11/dist-packages/accelerate/utils/__init__.py", line 14, in <module>
    from .ao import convert_model_to_fp8_ao, filter_first_and_last_linear_layers, has_ao_layers
  File "/usr/local/lib/python3.11/dist-packages/accelerate/utils/ao.py", line 28, in <module>
    from torchao.float8.float8_linear import Float8LinearConfig
  File "/usr/local/lib/python3.11/dist-packages/torchao/__init__.py", line 41, in <module>
    from torchao.quantization import (
  File "/usr/local/lib/python3.11/dist-packages/torchao/quantization/__i

In [25]:
!ls $TRAIN_DIR/target

mask.png  target.png


In [24]:
# See the output model
!ls $OUTPUT_DIR

checkpoint-100	 checkpoint-1600  checkpoint-400     logs
checkpoint-1000  checkpoint-1700  checkpoint-500     model_index.json
checkpoint-1100  checkpoint-1800  checkpoint-600     scheduler
checkpoint-1200  checkpoint-1900  checkpoint-700     text_encoder
checkpoint-1300  checkpoint-200   checkpoint-800     tokenizer
checkpoint-1400  checkpoint-2000  checkpoint-900     unet
checkpoint-1500  checkpoint-300   feature_extractor  vae


In [6]:
# For inference

!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

2025-04-24 14:47:09.176446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745506029.416794     108 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745506029.490238     108 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Fetching 13 files:   0%|                                 | 0/13 [00:00<?, ?it/s]
model.safetensors:   0%|                            | 0.00/1.36G [00:00<?, ?B/s]

merges.txt:   0%|                                    | 0.00/525k [00:00<?, ?B/s]


preprocessor_config.json: 100%|████████████████| 342/342 [00:00<00:00, 2.21MB/s]



Fetching 13 files:   8%|█▉                       | 1/13 [00:00<00:02,  4.59it/s]



scheduler_config.json: 

In [30]:
!ls

# !mv flowerwoman-model/tokenizer/tokenizer_config.json flowerwoman-model/tokenizer/config.json

APAI3010_Realfill    infer.py	realfill.zip		 train_realfill.py
data		     LICENSE	requirements_kaggle.txt
flowerwoman-model    model.zip	requirements.txt
flowerwoman-results  README.md	train_realfill.ipynb


In [18]:
# Writing to file on Kaggle

content = '''import argparse
import os

import torch
from PIL import Image, ImageFilter
from diffusers import (
    StableDiffusionInpaintPipeline, 
    UNet2DConditionModel,
    DDPMScheduler
)
from transformers import CLIPTextModel

parser = argparse.ArgumentParser(description="Inference")
parser.add_argument(
    "--model_path",
    type=str,
    default=None,
    required=True,
    help="Path to pretrained model or model identifier from huggingface.co/models.",
)
parser.add_argument(
    "--validation_image",
    type=str,
    default=None,
    required=True,
    help="The directory of the validation image",
)
parser.add_argument(
    "--validation_mask",
    type=str,
    default=None,
    required=True,
    help="The directory of the validation mask",
)
parser.add_argument(
    "--output_dir",
    type=str,
    default="./test-infer/",
    help="The output directory where predictions are saved",
)
parser.add_argument("--seed", type=int, default=None, help="A seed for reproducible inference.")

args = parser.parse_args()

if __name__ == "__main__":
    os.makedirs(args.output_dir, exist_ok=True)
    generator = None 

    # create & load model
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting",
        torch_dtype=torch.float32,
        revision=None
    )

    pipe.unet = UNet2DConditionModel.from_pretrained(
        args.model_path, subfolder="unet", revision=None,
    )
    pipe.text_encoder = CLIPTextModel.from_pretrained(
        args.model_path, subfolder="text_encoder", revision=None,
    )
    pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to("cuda")

    if args.seed is not None:
        generator = torch.Generator(device="cuda").manual_seed(args.seed)
    
    image = Image.open(args.validation_image)
    mask_image = Image.open(args.validation_mask)

    erode_kernel = ImageFilter.MaxFilter(3)
    mask_image = mask_image.filter(erode_kernel)

    blur_kernel = ImageFilter.BoxBlur(1)
    mask_image = mask_image.filter(blur_kernel)

    for idx in range(16):
        result = pipe(
            prompt="a photo of sks", image=image, mask_image=mask_image, 
            num_inference_steps=200, guidance_scale=1, generator=generator,
            height=image.size[1], width=image.size[0],
        ).images[0]
        
        result = Image.composite(result, image, mask_image)
        result.save(f"{args.output_dir}/{idx}.png")

    del pipe
    torch.cuda.empty_cache()'''

def write_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    print(f"Content written to {filename}")

write_to_file("infer.py", content)

Content written to infer.py


In [8]:
!cat infer.py

import argparse
import os

import torch
from PIL import Image, ImageFilter
from diffusers import (
    StableDiffusionInpaintPipeline, 
    UNet2DConditionModel,
    DDPMScheduler
)
from transformers import CLIPTextModel

parser = argparse.ArgumentParser(description="Inference")
parser.add_argument(
    "--model_path",
    type=str,
    default=None,
    required=True,
    help="Path to pretrained model or model identifier from huggingface.co/models.",
)
parser.add_argument(
    "--validation_image",
    type=str,
    default=None,
    required=True,
    help="The directory of the validation image",
)
parser.add_argument(
    "--validation_mask",
    type=str,
    default=None,
    required=True,
    help="The directory of the validation mask",
)
parser.add_argument(
    "--output_dir",
    type=str,
    default="./test-infer/",
    help="The output directory where predictions are saved",
)
parser.add_argument("--seed", type=int, default=None, help="A seed for reproducible inference.")